In [1]:
def run_bls(time, flux, 
            bls_params: dict = {'min_per':0.5, 'max_per':15, 
                                'minimum_n_transit':2, 
                                'freq_factor':1,
                                'durations':[0.05, 0.06666666666666668, 
                                             0.08333333333333334, 0.1,
                                             0.11666666666666668, 
                                             0.13333333333333336,
                                             0.15000000000000002, 
                                             0.16666666666666669, 
                                             0.18333333333333335, 0.2], 
                                'objective':'snr'}, 
            compute_stats: bool = True): 

    '''
    args: 
        stitched_lc: list of stitched light curve arrays [time, flux]
        bls_params: params for bls execution. see documentation
        compute_stats: compute statistics on best period/duration combination? default False
    returns: 
        best_params: [index, period, t0, duration, sig_diff] for highest power period (sig_diff is sig_diff between left/right depths)
        results: the BLS results array 
        bls_model: the BLS model  
        in_transit_mask: mask for the in_transit points. to get not in transit, do ~in_transit_mask
        stats: if compute_stats==True, then the stats on the best period/duration/t0 are returned
    '''

    from astropy.timeseries import BoxLeastSquares
    import numpy as np

    durations = np.array(bls_params['durations'])
    bls_model = BoxLeastSquares(t=time, y=flux)
    results = bls_model.autopower(durations, frequency_factor=bls_params['freq_factor'], 
                            minimum_period=bls_params['min_per'], 
                            maximum_period=bls_params['max_per'],
                            objective=bls_params['objective'])

    index = np.argmax(results.power)
    period = results.period[index]
    t0 = results.transit_time[index]
    duration = results.duration[index]
    in_transit = bls_model.transit_mask(time, period, 2*duration, t0)

    best_params = [index, period, t0, duration]

    if compute_stats: 
        stats = bls_model.compute_stats(period, duration, t0)

        depth_odd = stats['depth_odd']
        err_odd = depth_odd[1]
        depth_odd = depth_odd[0]
        depth_even = stats['depth_even']
        err_even = depth_even[1]
        depth_even = depth_even[0]

        diff = np.abs(depth_odd-depth_even)
        unc_diff = ((err_even/depth_even)**2+(err_odd/depth_odd)**2)**0.5
        sig_diff = diff/unc_diff 

        best_params.append(sig_diff)

        return best_params, results, bls_model, in_transit, stats

    else: 
        return best_params, results, bls_model, in_transit 

In [7]:

import numpy as np
def bls_validation_mosaic(tic_id:str, clean_time:np.array, detrend_flux:np.array, 
                          raw_time:np.array, raw_flux:np.array, 
                          best_params:list, bls_results, bls_model, in_transit, bls_stats, 
                          path:str=None, dpi:int=150): 

    '''
    arguments: 
        tic_id: tic id 
        clean_time: detrended and flare removed time array
        detrend_flux: flux values corresponding to clean_time arg
        raw_time: array of "raw" time values, i.e. not detrended and potentially with flares
        raw_flux: flux array corresponding to raw_time arg
        best_params, bls_results, bls_model, in_transit, bls_stats: the items returned by the run_bls function in pipeline_functions.py (NOTE: in run_bls, stats must be set to be calculated!)
        path: if defined, plot will be saved as the provided path. Otherwise, it will be displayed
        dpi: dpi of saved plot

    returns: 
    '''

    import matplotlib.pyplot as plt
    from matplotlib.gridspec import GridSpec
    from sunnyhills.borrowed import tls_intransit_stats
    from sunnyhills.misc import phase, rebin_lightcurve
    from lightkurve.periodogram import Periodogram

    plt.style.use('https://raw.githubusercontent.com/thissop/MAXI-J1535/main/code/misc/stolen_science.mplstyle?token=GHSAT0AAAAAABP54PQO2X2VXMNS256IWOBOYRNCFBA')
    fig = plt.figure(constrained_layout=True, figsize=(12,12))

    gs = GridSpec(4, 3, figure=fig)
    ax1 = fig.add_subplot(gs[0, :])
    ax2 = fig.add_subplot(gs[1, :])
    ax3 = fig.add_subplot(gs[2, :-1])
    ax4 = fig.add_subplot(gs[-1, 0])
    ax5 = fig.add_subplot(gs[-1, -2])
    ax6 = fig.add_subplot(gs[2:, -1])

    
    # raw and trend light curve
    p = Periodogram(bls_results.period,bls_model.power)
    p.flatten()
    p.plot(ax=ax1,xlabel='period',ylabel='power',style='https://raw.githubusercontent.com/thissop/MAXI-J1535/main/code/misc/stolen_science.mplstyle?token=GHSAT0AAAAAABP54PQO2X2VXMNS256IWOBOYRNCFBA')
    '''
    ax1.scatter(raw_time, raw_flux, s=1)
    ax1.set(ylabel='Flux')
    '''

    # detrend light curve
    ax2.scatter(clean_time, detrend_flux, s=1)
    index = np.argmax(bls_results.power)
    period = bls_results.period[index]
    t0 = bls_results.transit_time[index]
    duration = bls_results.duration[index]


    phased_time, phased_flux, x, f = phase(clean_time, detrend_flux, best_params, bls_model)

    ax2.vlines(clean_time[in_transit], min(detrend_flux), max(detrend_flux), color='red', lw=0.05,alpha=0.4, zorder=0)
    ax2.set(ylabel='Detrended Flux')

    for ax in [ax1, ax2]: 
        ax.set(xlabel='Time (days)')

    # phase folded
    ax3.scatter(phased_time, phased_flux, s=3, c='grey')

    binned_x, binned_flux = rebin_lightcurve(phased_time, phased_flux)
    
    ax3.plot(x, f, color='red', alpha=0.5)
    ax3.scatter(binned_x, binned_flux, c='orange', s=40, edgecolor='black')

    ax3.set(xlim=(-0.2, 0.2))

    for ax in [ax3, ax4, ax5]: 
        ax.set(xlabel='Time from mid-transit (days)', ylabel='Detrended Flux')

    # transit depths (odd, even)

    # https://github.com/hippke/tls/blob/71da590d3e199264822db425ab9f9f633253986e/transitleastsquares/stats.py#L338

    sig_diff = best_params[4]

    intransit_stats = tls_intransit_stats(clean_time, detrend_flux, 
                                          bls_stats['transit_times'], 
                                          best_params[3])

    odd_ = intransit_stats[4]
    odd_time, odd_flux = (odd_[0], odd_[1])
    odd_phased_time, odd_phased_flux, odd_x, odd_f = phase(odd_time, odd_flux, best_params, bls_model)
    odd_binned_time, odd_binned_flux = rebin_lightcurve(odd_phased_time, odd_phased_flux, factor=20)

    even_ = intransit_stats[5]
    even_time, even_flux = (even_[0], even_[1])
    even_phased_time, even_phased_flux, even_x, even_f = phase(even_time, even_flux, best_params, bls_model)
    even_phased_time = even_phased_time + 2.5*np.max(odd_phased_time) # shift over to show together 
    
    odd_even_median = (np.max(odd_phased_time)+np.min(even_phased_time))/2

    even_binned_time, even_binned_flux = rebin_lightcurve(even_phased_time, even_phased_flux, factor=20)

    ax4.scatter(odd_phased_time, odd_phased_flux, s=3, c='grey')
    ax4.scatter(even_phased_time, even_phased_flux, s=3, c='grey')
    ax4.scatter(odd_binned_time, odd_binned_flux, c='orange', s=40, edgecolor='black')
    ax4.scatter(even_binned_time, even_binned_flux, c='orange', s=40, edgecolor='black')

    ax4.axvline(x=odd_even_median, color='black', lw=0.5, label='Diff: '+str(round(sig_diff, 5))+r'$\sigma$')

    ax4.legend(loc='upper right', handlelength=0)

    ax4.set(xlabel='Odd (left) and Even (right) Folded Transits')
    ax4.xaxis.set_major_locator(plt.NullLocator())

    # periodogram 
    ax5.plot(bls_results.period, bls_results.power)
    ax5.set(xlabel='Period (d)', ylabel='Power')

    #ax6.axis('off')
    ax6.tick_params(labelbottom=False, labelleft=False, axis='both', which='both', length=0)

    index = np.argmax(bls_results.power)
    text_info = []
    for key_name in bls_results.keys(): 
        result = bls_results[key_name]
        if type(result)!=str: 
            result = str(round(result[index], 5))

        text_info.append(key_name+': '+result+'/n')

    ax6.text(x=0.1, y=0.5, s='/n'.join(str(i).replace('_','') for i in text_info), fontsize='large', va='center', transform=ax6.transAxes)

    ax1.set_title('TIC: '+str(tic_id).replace('_','')+' PERIOD: '+str(round(period, 5)), size='xx-large')

    if path==None:
        plt.show()
    else: 
        plt.savefig(path, dpi=dpi)

In [8]:
import pandas as pd

lc_csv = pd.read_csv('C:/Users/60002/Documents/GitHub/sunnyhills/data/current/processed/two_min_lightcurves--temp/TIC_1232360.csv')

flux = lc_csv['detrended_flux']
time = lc_csv['cleaned_time']

<bound method NDFrame.head of        cleaned_time  detrended_flux   trend_time  trend_flux     raw_time  \
0       1517.936742        1.006893  1517.936742    0.999730  1517.936742   
1       1517.940909        1.006908  1517.940909    0.999686  1517.938131   
2       1517.943687        1.004701  1517.943687    0.999638  1517.939520   
3       1517.946465        1.005554  1517.946465    0.999604  1517.940909   
4       1517.950631        1.006714  1517.950631    0.999566  1517.942298   
...             ...             ...          ...         ...          ...   
26534           NaN             NaN          NaN         NaN  2279.977849   
26535           NaN             NaN          NaN         NaN  2279.979238   
26536           NaN             NaN          NaN         NaN  2279.980627   
26537           NaN             NaN          NaN         NaN  2279.982016   
26538           NaN             NaN          NaN         NaN  2279.983405   

       raw_flux  
0      1.006621  
1      1.